# <font color = 'blue'>Data Scrapping and Data Cleansing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,OrdinalEncoder,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from datetime import datetime
import urllib.request
from bs4 import BeautifulSoup
import json
from urllib.request import Request, urlopen
import numpy as np
import pandas as pd
import ssl
import requests
from statistics import mean
import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Defining Areas 

areas = ['Alameda','Alamo',
'Atherton',
'Belmont',
'Berkeley',
'Burlingame',
'Campbell',
'Canyon',
'Castro Valley',
'Cupertino',
'Daly City',
'Danville',
'Dublin',
'El Cerrito',
'Fremont',
'Half Moon Bay',
'Hayward',
'Lafayette',
'Los Altos',
'Los Gatos',
'Menlo Park',
'Mill Valley',
'Millbrae',
'Milpitas',
'Mountain View',
'Newark',
'Oakland',
'Orinda',
'Pacifica',
'Palo Alto',
'Pleasanton',
'Redwood City',
'Richmond',
'San Bruno',
'San Carlos',
'San Francisco',
'San Francisco',
'San Jose',
'San Leandro',
'San Lorenzo',
'San Mateo',
'San Pablo',
'San Quentin',
'San Rafael',
'San Ramon',
'Santa Clara',
'Saratoga',
'Sausalito',
'South San Francisco',
'Stinson Beach',
'Sunnyvale',
'Tiburon',
'Union City',
'Walnut Creek']

In [ ]:
# Defining a function to get soup object

def gethtml(url):
    
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    
    header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,}
    
    # Making the website believe that we are accessing website using a mozilla browser
    req = Request(url, headers=header)
    webpage = urlopen(req).read()

    # Creating a BeautifulSoup object and parsing data with lxml
    soup = BeautifulSoup(webpage, 'html.parser')
    return soup

In [ ]:
# Getting all uniques links for each house and saving in a data frame

zillow_df = np.nan
zillow_detail_df = True
def zillow(url,z_df = pd.DataFrame()):
    soup = gethtml(url)
    d_json = soup.find_all('script',attrs={'data-zrr-shared-data-key':"mobileSearchPageStore"})[0].string[4:-3].encode('utf8')
    data = json.loads(d_json)
    try:
        z_df = pd.DataFrame(data['cat1']['searchResults']['listResults']).loc[:,['zpid','detailUrl','hdpData']]
    except:
        z_df = z_df
    
    return z_df

In [ ]:
'''Here user can select number pages that are required to scrape for each area.If 2 is selected then 2 pages 
of listimgs will be scrapped'''

count = 0
pages = int(input("Select number of pages to scrape:"))
print("No of Pages to scrape Data from ",pages)
for i in areas:
    i = i.replace(" ","%20") + "-CA"
    for p in range(pages):
        link = f'https://www.zillow.com/homes/{i}/{p+1}_p'
        print(link)
        time.sleep(2)
        if count==0:
            zillow_df = zillow(link)
        else:
            zillow_df = pd.concat([zillow_df,zillow(link)],axis=0)
        count = count+1

In [ ]:
print(gethtml("https://www.zillow.com/homes/Oakland-CA/1_p"))

In [ ]:
# Removing duplicates
zillow_df.drop_duplicates(subset = ['zpid'],keep = 'first',inplace = True)

In [ ]:
# Getting house data in zillow file
c=0
for i in zillow_df['hdpData']:
    if c ==0:
        c=c+1
        zillow_detail_df = pd.DataFrame(i['homeInfo'])
    else:
        zillow_detail_df = zillow_detail_df.append(pd.DataFrame(i['homeInfo']),ignore_index=True)
                                     
zillow_detail_df.drop_duplicates(subset = ['zpid'],keep = 'first',inplace = True)
zillow_detail_df.reset_index(drop = True,inplace=True)

In [ ]:
# List of fact columns that are required for analysis
cols = ['BuildingType',
 'CalendarYear built',
 'HeatingHeating',
 'SnowflakeCooling',
 'ParkingParking',
 'HOAHOA',
 'Price Square FeetPrice/sqft',
 'Bedrooms',
 'Bathrooms',
 'Full bathrooms',
 'Features',
 'Flooring',
 'Heating features',
 'Cooling features',
 'Appliances included',
 'Laundry features',
 'Window features',
 'Interior features',
 'Total structure area',
 'Total interior livable area',
 'Total number of fireplaces',
 'Fireplace features',
 'Total spaces',
 'Parking features',
 'Garage spaces',
 'Covered spaces',
 'Accessibility features',
 'Levels',
 'Stories',
 'Entry location',
 'Pool features',
 'Exterior features',
 'Patio and porch details',
 'View description',
 'Lot features',
 'Special conditions',
 'Home type',
 'Architectural style',
 'Property subType',
 'Construction materials',
 'Roof',
 'Property condition',
 'New construction',
 'Year built',
 'Builder name',
 'Electric information',
 'Sewer information',
 'Water information',
 'Security features',
 'Region',
 'Subdivision',
 'HOA fee',
 'Amenities included',
 'Services included',
 'Association name',
 'Association phone',
 'Price Range',
 'Listing Terms']


In [ ]:
# Scrapping data for facts data using house detail link
import datetime
print(datetime.datetime.now())
pricehistoty = pd.DataFrame()
facts = pd.DataFrame()#columns=cols)
f = {}
counter = 0
for info in zillow_df.loc[:,'detailUrl']:
    time.sleep(0.75)
    try:
        #zpid = info[-7::-1][:info[-7::-1].find("/")][::-1]
        soup = gethtml(info)
        attr = []
        vals =[]
        for k,v in json.loads(soup.find_all("script",id = "hdpApolloPreloadedData")[0].string).items():
            zpid = json.loads(v)[list(json.loads(v).keys())[1]]['property']['zpid']
            p = json.loads(v)[list(json.loads(v).keys())[1]]['property']['priceHistory']
            pricehistoty = pricehistoty.append(pd.DataFrame(p,index=[zpid]*(len(p))))
            break
            
        for j in soup.find_all(['li','span'],class_=['ListItem-c11n-8-48-0__sc-10e22w8-0 dgPprD','ds-home-fact-list-item',
                                                    'Text-c11n-8-48-0__sc-aiai24-0 dQezUG']):
            if j.text.split(":")[0] not in attr and j.text.split(":")[0] in cols:
                attr.append(j.text.split(":")[0])
                vals.append([j.text.split(":")[1]])
            f = zip(attr,vals)
        facts = facts.append(pd.DataFrame(dict(f),index=[zpid]),verify_integrity = True) 
    except:
        pass
    
    counter = counter +1
    print(facts.shape)
print(datetime.datetime.now())

In [ ]:
'''Defining a function and data frame to get the ratings data from area vibes'''

ratings = pd.DataFrame(columns=['Livability','Cost of Living','Crime','Employment','Housing','Schools','User Ratings'])
r = {}
def getratings(soup,area):
    for i in soup.find_all('nav',class_="category-menu-new"):
        print(i)
        x = i.find_all('em')
        y = i.find_all('i')
        for m in list(zip(x,y)):
            r.update({m[0].text:m[1].text})
    return r

In [ ]:
for i in areas:
    try:
        soup = gethtml(f'https://www.areavibes.com/{i.replace(" ","%20")}-ca/livability/')
        print(soup)
        ratings = ratings.append(pd.DataFrame(getratings(soup,i),index=[i]))
    except:
        pass
  
display(ratings)

In [ ]:
# Saving All the files
zillow_detail_df.to_csv("Zillow.csv")
facts.to_csv("facts.csv")
pricehistoty.to_csv("pricehistory.csv")
ratings.to_csv("ratings.csv")

In [ ]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 999

# <font color = 'blue'>Celaning Zillow Houses - File1

In [ ]:
#Reading data file
zillow = pd.read_csv("Zillow.csv")

In [ ]:
zillow.shape

(2686, 56)

### <font color = 'red'>1) Dropping unwanted columns from the data frame

In [ ]:
# Dropping all the columns that does not posses any valuble information
zillow.drop(labels=['homeStatus','imageLink','isFeatured','shouldHighlight','brokerId','listing_sub_type','openHouse','isUnmappable',
                   'mediumImageLink','isPreforeclosureAuction','homeStatusForHDP','open_house_info','priceForHDP',
                    'hiResImageLink','watchImageLink','tvImageLink','tvCollectionImageLink','tvHighResImageLink',
                    'desktopWebHdpImageLink','isNonOwnerOccupied','isPremierBuilder','isZillowOwned','currency','country','unit','videoCount',
                    'group_type','grouping_name','priceSuffix','providerListingID','newConstructionType','datePriceChanged',
                    'priceChange','priceReduction','isRentalWithBasePrice','comingSoonOnMarketDate'
                   ],axis=1,inplace=True)

In [ ]:
zillow.head()

,zpid,streetAddress,zipcode,city,state,latitude,longitude,price,bathrooms,bedrooms,livingArea,homeType,daysOnZillow,zestimate,rentZestimate,timeOnZillow,taxAssessedValue,lotAreaValue,lotAreaUnit,dateSold
0,24869311,965 Shorepoint Ct APT 204,94501,Alameda,CA,37.762020,-122.269200,419000.0,1.0,1.0,772.0,CONDO,0,451700.0,2163.0,1.632549e+12,184600.0,2.32461,acres,NaN
1,24860437,2057 San Jose Ave,94501,Alameda,CA,37.764296,-122.252671,1295000.0,4.0,4.0,3101.0,MULTI_FAMILY,0,1590400.0,4419.0,1.632541e+12,130505.0,7500.00000,sqft,NaN
2,24860197,2154 Santa Clara Ave,94501,Alameda,CA,37.767182,-122.247172,1798000.0,5.0,6.0,3175.0,SINGLE_FAMILY,0,1617900.0,2991.0,1.632538e+12,711168.0,10400.00000,sqft,1.034320e+12
3,24863829,612 Haight Ave,94501,Alameda,CA,37.774065,-122.278712,899000.0,2.0,2.0,1353.0,SINGLE_FAMILY,0,1041400.0,2027.0,1.632533e+12,742607.0,3744.00000,sqft,NaN
4,24856314,1109 Mound St,94501,Alameda,CA,37.756038,-122.238759,1150000.0,2.0,3.0,1760.0,SINGLE_FAMILY,0,1544200.0,3780.0,1.632481e+12,945261.0,4004.00000,sqft,1.187075e+12


### <font color = 'red'>2) Removing houses that has price greater than $8M$ and less than $0.1M$
    
##### We have choose 8Millions as thhreshold just to preserve the cities with higher prices like palo alto,Los Gatos

In [ ]:
zillow[['price']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2686 entries, 0 to 2685
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   2684 non-null   float64
dtypes: float64(1)
memory usage: 21.1 KB


In [ ]:
zillow[(zillow.price <8000000) & (zillow.price>100000)]

,zpid,streetAddress,zipcode,city,state,latitude,longitude,price,bathrooms,bedrooms,livingArea,homeType,daysOnZillow,zestimate,rentZestimate,timeOnZillow,taxAssessedValue,lotAreaValue,lotAreaUnit,dateSold
0,24869311,965 Shorepoint Ct APT 204,94501,Alameda,CA,37.762020,-122.269200,419000.0,1.0,1.0,772.0,CONDO,0,451700.0,2163.0,1.632549e+12,184600.0,2.32461,acres,NaN
1,24860437,2057 San Jose Ave,94501,Alameda,CA,37.764296,-122.252671,1295000.0,4.0,4.0,3101.0,MULTI_FAMILY,0,1590400.0,4419.0,1.632541e+12,130505.0,7500.00000,sqft,NaN
2,24860197,2154 Santa Clara Ave,94501,Alameda,CA,37.767182,-122.247172,1798000.0,5.0,6.0,3175.0,SINGLE_FAMILY,0,1617900.0,2991.0,1.632538e+12,711168.0,10400.00000,sqft,1.034320e+12
3,24863829,612 Haight Ave,94501,Alameda,CA,37.774065,-122.278712,899000.0,2.0,2.0,1353.0,SINGLE_FAMILY,0,1041400.0,2027.0,1.632533e+12,742607.0,3744.00000,sqft,NaN
4,24856314,1109 Mound St,94501,Alameda,CA,37.756038,-122.238759,1150000.0,2.0,3.0,1760.0,SINGLE_FAMILY,0,1544200.0,3780.0,1.632481e+12,945261.0,4004.00000,sqft,1.187075e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,2068661063,1954 Trinity Ave #203,94596,Walnut Creek,CA,37.901621,-122.068919,1679000.0,2.0,2.0,1464.0,CONDO,24,1679000.0,3879.0,1.630384e+12,NaN,NaN,NaN,NaN
2682,2075861524,2724 Oak Rd APT 90,94597,Walnut Creek,CA,37.922285,-122.056599,380000.0,1.0,1.0,620.0,CONDO,26,NaN,NaN,1.630206e+12,NaN,NaN,NaN,1.556608e+12
2683,18384673,1143 Terrebonne Dr,94598,Walnut Creek,CA,37.930112,-122.035782,1175000.0,3.0,4.0,2058.0,SINGLE_FAMILY,27,1233800.0,5003.0,1.630191e+12,699444.0,9430.00000,sqft,1.016179e+12
2684,18410650,529 Pimlico Ct,94597,Walnut Creek,CA,37.912314,-122.062668,690000.0,3.0,2.0,1185.0,TOWNHOUSE,27,690000.0,2949.0,1.630144e+12,571365.0,1500.00000,sqft,1.086332e+12


In [ ]:
zillow = zillow[(zillow.price <8000000) & (zillow.price>100000)]

In [ ]:
zillow[['price']].describe()

,price
count,2.593000e+03
mean,1.572115e+06
std,1.271487e+06
min,1.050000e+05
25%,7.890000e+05
50%,1.198000e+06
75%,1.800000e+06
max,7.998000e+06


### <font color = 'red'>3) Checking the percentage of null values in each column
- Filtering all the columns that has more than **50%** of data as null.

In [ ]:
zillow.isna().mean()

zpid                0.000000
streetAddress       0.000000
zipcode             0.000000
city                0.000000
state               0.000000
latitude            0.012341
longitude           0.012341
price               0.000000
bathrooms           0.094099
bedrooms            0.083687
livingArea          0.067104
homeType            0.000000
daysOnZillow        0.000000
zestimate           0.253374
rentZestimate       0.094099
timeOnZillow        0.075588
taxAssessedValue    0.218280
lotAreaValue        0.165831
lotAreaUnit         0.165831
dateSold            0.376398
dtype: float64

In [ ]:
zillow = zillow.loc[:,zillow.isna().mean()<0.50]

In [ ]:
zillow[zillow.homeType == 'MANUFACTURED']

,zpid,streetAddress,zipcode,city,state,latitude,longitude,price,bathrooms,bedrooms,livingArea,homeType,daysOnZillow,zestimate,rentZestimate,timeOnZillow,taxAssessedValue,lotAreaValue,lotAreaUnit,dateSold
307,2069963636,870 Camden Ave #37,95008,Campbell,CA,37.277421,-121.947425,269000.0,2.0,2.0,1000.0,MANUFACTURED,91,NaN,2782.0,1.624597e+12,NaN,NaN,NaN,NaN
308,2071493588,80 La Paz,95008,Campbell,CA,37.281484,-121.940347,324900.0,2.0,2.0,1040.0,MANUFACTURED,151,324900.0,3173.0,1.619477e+12,NaN,NaN,NaN,NaN
309,2071657075,31 Timber Cove Dr #31,95008,Campbell,CA,37.276492,-121.947397,309000.0,2.0,3.0,1344.0,MANUFACTURED,161,309000.0,3477.0,1.618556e+12,NaN,NaN,NaN,NaN
643,2068436645,711 Old Canyon Rd #114,94536,Fremont,CA,37.578338,-121.966668,148000.0,2.0,2.0,1100.0,MANUFACTURED,3,NaN,NaN,1.632205e+12,NaN,1100.0,sqft,NaN
666,54790318,711 Old Canyon Rd SPACE 110,94536,Fremont,CA,37.578338,-121.966668,318000.0,2.0,3.0,1547.0,MANUFACTURED,8,328600.0,1850.0,1.631774e+12,55404.0,1547.0,sqft,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2561,2107070853,162 Kona Cir,94587,Union City,CA,37.589025,-122.049802,229000.0,2.0,2.0,1440.0,MANUFACTURED,0,862900.0,2314.0,1.632545e+12,NaN,1440.0,sqft,1.432710e+12
2567,2068402513,2 Palm Dr,94587,Union City,CA,37.589510,-122.049221,154900.0,2.0,2.0,1020.0,MANUFACTURED,0,NaN,NaN,1.632459e+12,NaN,1020.0,sqft,NaN
2595,25059752,141 Polynesia Way,94587,Union City,CA,37.592401,-122.055194,197500.0,2.0,3.0,1440.0,MANUFACTURED,49,206000.0,2400.0,1.628260e+12,15469.0,NaN,NaN,NaN
2599,2097640387,148 Polynesia Way,94587,Union City,CA,37.592478,-122.054821,242000.0,2.0,2.0,1740.0,MANUFACTURED,67,251000.0,2400.0,1.626743e+12,NaN,NaN,NaN,1.472540e+12


### <font color = 'red'>4) Removing all rows with homeType as *LOT*

In [ ]:
zillow.homeType.value_counts()

SINGLE_FAMILY    1355
CONDO             603
TOWNHOUSE         275
LOT               165
MULTI_FAMILY      134
MANUFACTURED       61
Name: homeType, dtype: int64

In [ ]:
zillow = zillow[~zillow.homeType.isin(['LOT'])]

In [ ]:
zillow.homeType.value_counts().sum()

2428

### <font color = 'red'>5) Removing all rows that has latitude or longitue or price as null

In [ ]:
zillow[(zillow['latitude'].isna() | zillow['longitude'].isna())]

,zpid,streetAddress,zipcode,city,state,latitude,longitude,price,bathrooms,bedrooms,livingArea,homeType,daysOnZillow,zestimate,rentZestimate,timeOnZillow,taxAssessedValue,lotAreaValue,lotAreaUnit,dateSold
126,2069712373,12 Cardinal Ct,94025,Menlo Park,CA,NaN,NaN,4888888.0,5.0,5.0,2819.0,SINGLE_FAMILY,79,4894779.0,10967.0,1.625701e+12,NaN,6573.000000,sqft,NaN
449,15468440,(undisclosed Address),94014,Daly City,CA,NaN,NaN,1050000.0,3.0,5.0,1190.0,SINGLE_FAMILY,56,1173400.0,4235.0,1.627695e+12,NaN,2300.000000,sqft,NaN
887,2068436443,15009 Briggs Ct,95032,Los Gatos,CA,NaN,NaN,1598000.0,3.0,2.0,1508.0,CONDO,3,NaN,NaN,1.632207e+12,NaN,NaN,NaN,NaN
899,2068531025,16025 Bartlett Ct #186,95032,Los Gatos,CA,NaN,NaN,1893000.0,4.0,3.0,1966.0,CONDO,13,NaN,NaN,1.631357e+12,NaN,NaN,NaN,NaN
900,2068531209,16015 Bartlett Ct #181,95032,Los Gatos,CA,NaN,NaN,2103000.0,4.0,3.0,2089.0,CONDO,13,NaN,NaN,1.631353e+12,NaN,NaN,NaN,NaN
904,2068562757,16031 Bartlett Ct #189,95032,Los Gatos,CA,NaN,NaN,1958000.0,4.0,3.0,2040.0,CONDO,15,2046500.0,3433.0,1.631172e+12,NaN,NaN,NaN,NaN
921,2069509487,18860 Montalvo Oaks Cir,95030,Los Gatos,CA,NaN,NaN,3748000.0,6.0,5.0,3076.0,SINGLE_FAMILY,67,3930334.0,7669.0,1.626669e+12,NaN,6747.000000,sqft,NaN
936,2071615183,18858 Montalvo Oaks Cir,95030,Los Gatos,CA,NaN,NaN,3748000.0,5.0,4.0,3025.0,SINGLE_FAMILY,159,3782851.0,7207.0,1.618728e+12,NaN,3023.000000,sqft,NaN
1117,2068407465,305 Carlett Ter,94043,Mountain View,CA,NaN,NaN,1860000.0,3.0,3.0,1754.0,TOWNHOUSE,1,NaN,NaN,1.632444e+12,NaN,975.000000,sqft,NaN
1119,2068408497,2356 Toledo Ter,94043,Mountain View,CA,NaN,NaN,1640000.0,3.0,2.0,1465.0,TOWNHOUSE,1,NaN,NaN,1.632439e+12,NaN,1080.000000,sqft,NaN


In [ ]:
#Dropping rows that has latitue/longitude as null
zillow = zillow[(zillow.notnull()['latitude'] & zillow.notnull()['price'])]

In [ ]:
zillow.shape

(2406, 20)

### <font color = 'red'>6) Converting unix time(daysOnZillow,dateSold) to normal date 

In [ ]:
zillow.timeOnZillow = pd.to_datetime(zillow.timeOnZillow,unit = 'ms')
zillow.dateSold = pd.to_datetime(zillow.dateSold,unit='ms')

In [ ]:
zillow['timeOnZillow'].fillna(datetime.now(),inplace=True)

In [ ]:
zillow.isna().mean()

zpid                0.000000
streetAddress       0.000000
zipcode             0.000000
city                0.000000
state               0.000000
latitude            0.000000
longitude           0.000000
price               0.000000
bathrooms           0.032003
bedrooms            0.030341
livingArea          0.007897
homeType            0.000000
daysOnZillow        0.000000
zestimate           0.200748
rentZestimate       0.083541
timeOnZillow        0.000000
taxAssessedValue    0.180382
lotAreaValue        0.175810
lotAreaUnit         0.175810
dateSold            0.351205
dtype: float64

### <font color = 'red'>7) Replacing all the nulls in bathroom and bedroom columns with imputation (mode)

In [ ]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2406 entries, 0 to 2685
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   zpid              2406 non-null   int64         
 1   streetAddress     2406 non-null   object        
 2   zipcode           2406 non-null   int64         
 3   city              2406 non-null   object        
 4   state             2406 non-null   object        
 5   latitude          2406 non-null   float64       
 6   longitude         2406 non-null   float64       
 7   price             2406 non-null   float64       
 8   bathrooms         2329 non-null   float64       
 9   bedrooms          2333 non-null   float64       
 10  livingArea        2387 non-null   float64       
 11  homeType          2406 non-null   object        
 12  daysOnZillow      2406 non-null   int64         
 13  zestimate         1923 non-null   float64       
 14  rentZestimate     2205 n

In [ ]:
mode_imputer = SimpleImputer(strategy='most_frequent')

mode_columns = ['bedrooms','bathrooms']

for i in mode_columns:
    zillow[[i]] = mode_imputer.fit_transform(zillow[[i]])

In [ ]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2406 entries, 0 to 2685
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   zpid              2406 non-null   int64         
 1   streetAddress     2406 non-null   object        
 2   zipcode           2406 non-null   int64         
 3   city              2406 non-null   object        
 4   state             2406 non-null   object        
 5   latitude          2406 non-null   float64       
 6   longitude         2406 non-null   float64       
 7   price             2406 non-null   float64       
 8   bathrooms         2406 non-null   float64       
 9   bedrooms          2406 non-null   float64       
 10  livingArea        2387 non-null   float64       
 11  homeType          2406 non-null   object        
 12  daysOnZillow      2406 non-null   int64         
 13  zestimate         1923 non-null   float64       
 14  rentZestimate     2205 n

In [ ]:
mean_columns = ['livingArea','rentZestimate','taxAssessedValue'] 

mean_imputer = SimpleImputer(strategy='mean')

for i in mean_columns:
    zillow[[i]] = mode_imputer.fit_transform(zillow[[i]])

In [ ]:
zillow['zestimate'].fillna(zillow['price'],inplace=True)

In [ ]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2406 entries, 0 to 2685
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   zpid              2406 non-null   int64         
 1   streetAddress     2406 non-null   object        
 2   zipcode           2406 non-null   int64         
 3   city              2406 non-null   object        
 4   state             2406 non-null   object        
 5   latitude          2406 non-null   float64       
 6   longitude         2406 non-null   float64       
 7   price             2406 non-null   float64       
 8   bathrooms         2406 non-null   float64       
 9   bedrooms          2406 non-null   float64       
 10  livingArea        2406 non-null   float64       
 11  homeType          2406 non-null   object        
 12  daysOnZillow      2406 non-null   int64         
 13  zestimate         2406 non-null   float64       
 14  rentZestimate     2406 n

### <font color = 'red'>8) Removing outliers - houses that has bathrooms and bedrooms greater than 8
- <font color = 'black'>Converting bathrooms that has decimal values to closest floor values

In [ ]:
zillow = zillow[(zillow['bedrooms']<9) & (zillow['bathrooms']<9)]

In [ ]:
zillow['bedrooms'].value_counts()

3.0    847
2.0    637
4.0    514
5.0    156
1.0    151
6.0     56
7.0     18
0.0      9
8.0      7
Name: bedrooms, dtype: int64

In [ ]:
zillow['bathrooms'] = np.floor(zillow['bathrooms'])

In [ ]:
zillow.shape

(2395, 20)

### <font color = 'red'>9) Counting for number houses per city
- Dropping the cities that has less than 20 houses listed

In [ ]:
#Replacing city with name as Belvedere Tiburon to Tiburon as both are same
zillow['city'] = zillow.city.str.replace('Belvedere Tiburon','Tiburon')

In [ ]:
cities = zillow[['city','zpid']].groupby(by=['city']).count().reset_index()

In [ ]:
cities.sort_values(by=['zpid'],ascending=False)

,city,zpid
0,Alameda,80
44,San Mateo,80
40,San Francisco,79
53,Sunnyvale,79
56,Walnut Creek,79
41,San Jose,78
18,Fremont,78
4,Berkeley,77
48,Santa Clara,77
36,Redwood City,76


In [ ]:
#dropping cities that has less than 20 houses
zillow = zillow[zillow.city.isin(cities[cities['zpid']>=10]['city'])]

In [ ]:
zillow.shape

(2385, 20)

In [ ]:
zillow.head()

,zpid,streetAddress,zipcode,city,state,latitude,longitude,price,bathrooms,bedrooms,livingArea,homeType,daysOnZillow,zestimate,rentZestimate,timeOnZillow,taxAssessedValue,lotAreaValue,lotAreaUnit,dateSold
0,24869311,965 Shorepoint Ct APT 204,94501,Alameda,CA,37.762020,-122.269200,419000.0,1.0,1.0,772.0,CONDO,0,451700.0,2163.0,2021-09-25 05:53:23,184600.0,2.32461,acres,NaT
1,24860437,2057 San Jose Ave,94501,Alameda,CA,37.764296,-122.252671,1295000.0,4.0,4.0,3101.0,MULTI_FAMILY,0,1590400.0,4419.0,2021-09-25 03:28:24,130505.0,7500.00000,sqft,NaT
2,24860197,2154 Santa Clara Ave,94501,Alameda,CA,37.767182,-122.247172,1798000.0,5.0,6.0,3175.0,SINGLE_FAMILY,0,1617900.0,2991.0,2021-09-25 02:38:20,711168.0,10400.00000,sqft,2002-10-11 07:00:00
3,24863829,612 Haight Ave,94501,Alameda,CA,37.774065,-122.278712,899000.0,2.0,2.0,1353.0,SINGLE_FAMILY,0,1041400.0,2027.0,2021-09-25 01:23:17,742607.0,3744.00000,sqft,NaT
4,24856314,1109 Mound St,94501,Alameda,CA,37.756038,-122.238759,1150000.0,2.0,3.0,1760.0,SINGLE_FAMILY,0,1544200.0,3780.0,2021-09-24 11:03:11,945261.0,4004.00000,sqft,2007-08-14 07:00:00


In [ ]:
zillow.homeType.value_counts()

SINGLE_FAMILY    1330
CONDO             596
TOWNHOUSE         271
MULTI_FAMILY      128
MANUFACTURED       60
Name: homeType, dtype: int64

# <font color = 'red'>10) Changing lotareavlaue from acres to sqft and replacing nulls

In [ ]:
zillow[['lotAreaValue','lotAreaUnit']]

,lotAreaValue,lotAreaUnit
0,2.32461,acres
1,7500.00000,sqft
2,10400.00000,sqft
3,3744.00000,sqft
4,4004.00000,sqft
...,...,...
2681,NaN,NaN
2682,NaN,NaN
2683,9430.00000,sqft
2684,1500.00000,sqft


In [ ]:
zillow.loc[zillow.lotAreaUnit == 'acres','lotAreaValue']=zillow['lotAreaValue'][zillow.lotAreaUnit == 'acres']*43560

In [ ]:
zillow.loc[zillow.lotAreaUnit == 'acres','lotAreaUnit'] = 'sqft'

In [ ]:
zillow[['lotAreaValue']] = mean_imputer.fit_transform(zillow[['lotAreaValue']])

In [ ]:
zillow.drop('lotAreaUnit',inplace=True,axis=1)

# <font color='blue'>Cleaning Facts data - File 2

In [ ]:
facts = pd.read_csv("facts.csv")

## <font color = 'red'>1)Dropping Unwanted columns

In [ ]:
facts.drop(['HOAHOA','Full bathrooms','Interior features','Fireplace features','Special conditions',
           'Home type','Architectural style','Property subType','Construction materials','Roof','New construction'
            ,'Electric information','Region','Subdivision','HOA fee','Amenities included','Services included','Year built',
            'Association name','Association phone','Price Range'
           ,'Listing Terms','Appliances included','Patio and porch details','Sewer information',
           'Window features','Accessibility features','View description','Builder name','Entry location'],inplace = True,axis = 1)

### <font color = 'red'>2)Replacing Values in HasHeating,HasColling,HasFlooring,HasLaundary,HasPool,HasSecurity with either Yes or no

In [ ]:
facts['HasHeating'] = facts['HeatingHeating'].apply(lambda x: 'Yes' if x not in ['No Data',
                                                                                 'None',
                                                                                 'See Remarks',np.NAN] else 'No')

facts['HasCooling'] = facts['SnowflakeCooling'].apply(lambda x: 'Yes' if x not in ['None',
                                                                                   'No Data',
                                                                                   'No Air Conditioning',
                                                                                   'See Remarks',
                                                                                   'No Air Conditioning, None',
                                                                                   'None, See Remarks',np.NAN] else 'No')

facts['HasFlooring'] = facts['Flooring'].apply(lambda x: 'Yes' if x not in ['None','See Remarks',np.NAN] else 'No')

facts['HasLaundary'] = facts['Laundry features'].apply(lambda x: 'Yes' if x not in ['None','See Remarks',np.NAN] else 'No')

facts['HasPool'] = facts['Pool features'].apply(lambda x: 'Yes' if x not in ['None','See Remarks',np.NAN] else 'No')

facts['HasSecurity'] = facts['Security features'].apply(lambda x: 'Yes' if x not in ['None','See Remarks',np.NAN] else 'No')


In [ ]:
# Replacing nulls in stories,fireplaces with 0 and Converting garage spaces values from aplhanumerical to numeric

In [ ]:
facts['Stories'] = facts.Stories.fillna(value=0)

facts['GarageSpaces'] = facts.ParkingParking.apply(lambda x : int(x[0:2]) if (x[0].isdigit() or x[0].isdigit())  else 0)
facts.loc[(facts['GarageSpaces']==0) & facts.ParkingParking.str.contains('Garage - Attached'),'GarageSpaces'] = 1

facts['FirePlaces'] = facts['Total number of fireplaces'].fillna(0)

## <font color='red'>3)Converting Price.Sqft,calender year built and living area from string to int

In [ ]:
facts['Price Square FeetPrice/sqft'] = facts['Price Square FeetPrice/sqft'].str.strip('$').str.replace(",","")
pd.to_numeric(facts['Price Square FeetPrice/sqft'],errors = 'coerce')
facts[['Price Square FeetPrice/sqft']] = mean_imputer.fit_transform(facts[['Price Square FeetPrice/sqft']])

In [ ]:
facts['Total interior livable area'] = facts['Total interior livable area'].str.strip('sqft').str.replace(",","")
pd.to_numeric(facts['Total interior livable area'],errors = 'coerce')
facts[['Total interior livable area']] = mean_imputer.fit_transform(facts[['Total interior livable area']])

In [ ]:
facts['CalendarYear built']

0          1971
1       No Data
2          1909
3          1910
4          1906
         ...   
2677       2021
2678       1972
2679       1964
2680       1985
2681       1989
Name: CalendarYear built, Length: 2682, dtype: object

In [ ]:
facts['CalendarYear built']=pd.to_numeric(facts['CalendarYear built'],errors = 'coerce')
facts[['CalendarYear built']] = mode_imputer.fit_transform(facts[['CalendarYear built']])
facts['CalendarYear built']

0       1971.0
1       2021.0
2       1909.0
3       1910.0
4       1906.0
         ...  
2677    2021.0
2678    1972.0
2679    1964.0
2680    1985.0
2681    1989.0
Name: CalendarYear built, Length: 2682, dtype: float64

In [ ]:
# Choosing columns that are required for analysis
facts = facts [['Zpid','CalendarYear built','Price Square FeetPrice/sqft',
              'HasFlooring','HasHeating','HasCooling','GarageSpaces','HasLaundary',
                'FirePlaces','HasPool','HasSecurity','Stories']]

# <font color = 'blue'>Cleaning Ratings Data - File 3 

In [ ]:
ratings = pd.read_csv('Ratings.csv')
ratings.fillna('B',inplace=True)
ratings.drop_duplicates(subset=ratings.columns,inplace = True)
ratings.drop(['Cost of Living','Amenities','User Ratings'],inplace=True,axis = 1)
ratings.head()

,city,Livability,Crime,Employment,Housing,Schools
0,Alameda,76,C,B,B-,B+
1,Alamo,82,A,A+,D,A+
2,Atherton,78,A-,A+,D,A+
3,Belmont,81,A-,A,C-,A+
4,Berkeley,64,F,B-,D,B-


In [ ]:
main_df = pd.merge(left=zillow,right=facts,how='inner',left_on='zpid',right_on='Zpid',copy=False)

In [ ]:
main_df = pd.merge(left=main_df,right=ratings,how='left',left_on = 'city',right_on='city')

In [ ]:
main_df.bathrooms = main_df.bathrooms.astype(np.int64)
main_df.bedrooms = main_df.bedrooms.astype(np.int64)
main_df['CalendarYear built'] = main_df['CalendarYear built'].astype(np.int64)
main_df.FirePlaces = main_df.FirePlaces.astype(np.int64)
main_df.Stories = main_df.Stories.astype(np.int64)

In [ ]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2381 entries, 0 to 2380
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   zpid                         2381 non-null   int64         
 1   streetAddress                2381 non-null   object        
 2   zipcode                      2381 non-null   int64         
 3   city                         2381 non-null   object        
 4   state                        2381 non-null   object        
 5   latitude                     2381 non-null   float64       
 6   longitude                    2381 non-null   float64       
 7   price                        2381 non-null   float64       
 8   bathrooms                    2381 non-null   int64         
 9   bedrooms                     2381 non-null   int64         
 10  livingArea                   2381 non-null   float64       
 11  homeType                     2381 non-null 

In [ ]:
# Dropping duplicated column Zpid
main_df.drop('Zpid',axis=1,inplace=True)

In [ ]:
# Saving Data to main_df and using this file for analysis
main_df.to_csv("main_df.csv",index=False)

In [ ]:
main_df.set_index('zpid',inplace=True)